In [ ]:
# Cell 1: Import libraries
import os
import json
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.optim as optim

In [ ]:
# Cell 2: Define the dataset for JSON keypoint files (updated to enforce fixed frame shape)
class ASLDataset(Dataset):
    def __init__(self, json_dir, max_seq_length=100, default_num_keypoints=31, default_keypoint_dim=4, transform=None):
        self.json_files = [os.path.join(json_dir, f) for f in os.listdir(json_dir) if f.endswith('.json')]
        self.max_seq_length = max_seq_length
        self.default_num_keypoints = default_num_keypoints
        self.default_keypoint_dim = default_keypoint_dim
        self.transform = transform
        # Extract unique labels from filenames (assumes "label_videoId.json" naming)
        self.labels = sorted(list({self._get_label(f) for f in self.json_files}))
        self.label_to_idx = {label: idx for idx, label in enumerate(self.labels)}
    
    def _get_label(self, filepath):
        filename = os.path.basename(filepath)
        return filename.split('_')[0]
    
    def _pad_or_trim(self, keypoints):
        # keypoints: (num_frames, num_keypoints, keypoint_dim)
        num_frames = keypoints.shape[0]
        if num_frames < self.max_seq_length:
            pad_shape = (self.max_seq_length - num_frames, *keypoints.shape[1:])
            padding = np.zeros(pad_shape, dtype=keypoints.dtype)
            keypoints = np.concatenate([keypoints, padding], axis=0)
        elif num_frames > self.max_seq_length:
            keypoints = keypoints[:self.max_seq_length]
        return keypoints

    def process_frame(self, frame):
        """
        Process a single frame.
        If the frame is a dict with a "pose" key, convert that value to a NumPy array,
        then enforce the shape (default_num_keypoints, default_keypoint_dim) by trimming or padding.
        """
        if isinstance(frame, dict):
            if "pose" in frame and frame["pose"] is not None:
                pose = np.array(frame["pose"])
            else:
                pose = np.zeros((self.default_num_keypoints, self.default_keypoint_dim))
        else:
            pose = np.array(frame)
        
        # Ensure the pose array has two dimensions: (num_keypoints, keypoint_dim)
        if pose.ndim != 2:
            # If not, attempt to reshape or default to zeros
            pose = np.zeros((self.default_num_keypoints, self.default_keypoint_dim))
        
        # If the number of columns is different from expected, trim or pad the keypoint dimension
        n, d = pose.shape
        if d < self.default_keypoint_dim:
            pad_d = np.zeros((n, self.default_keypoint_dim - d))
            pose = np.concatenate([pose, pad_d], axis=1)
        elif d > self.default_keypoint_dim:
            pose = pose[:, :self.default_keypoint_dim]
        
        # Enforce a fixed number of keypoints by padding or trimming rows
        if n < self.default_num_keypoints:
            pad_rows = np.zeros((self.default_num_keypoints - n, self.default_keypoint_dim))
            pose = np.concatenate([pose, pad_rows], axis=0)
        elif n > self.default_num_keypoints:
            pose = pose[:self.default_num_keypoints, :]
        
        return pose
    
    def __len__(self):
        return len(self.json_files)
    
    def __getitem__(self, idx):
        json_path = self.json_files[idx]
        with open(json_path, 'r') as f:
            data = json.load(f)
        # If keypoints are missing or empty, create a dummy sequence with zeros.
        if 'keypoints' not in data or not data['keypoints']:
            keypoints = np.zeros((self.max_seq_length, self.default_num_keypoints, self.default_keypoint_dim))
        else:
            raw_keypoints = data['keypoints']
            # Process if keypoints are stored as a dict (frame-indexed)
            if isinstance(raw_keypoints, dict):
                sorted_frame_keys = sorted(raw_keypoints.keys(), key=lambda x: int(x) if x.isdigit() else x)
                frame_list = [self.process_frame(raw_keypoints[k]) for k in sorted_frame_keys]
                keypoints = np.array(frame_list)
            # If keypoints are stored as a list, process each frame
            elif isinstance(raw_keypoints, list):
                frame_list = [self.process_frame(frame) for frame in raw_keypoints]
                keypoints = np.array(frame_list)
            else:
                keypoints = np.array(raw_keypoints)
        
        # Apply padding or trimming along the temporal dimension.
        keypoints = self._pad_or_trim(keypoints)
        
        # Normalize coordinates if keypoints are non-zero.
        if np.any(keypoints):
            max_val = np.max(np.abs(keypoints))
            if max_val > 0:
                keypoints = keypoints / max_val
        
        label = self._get_label(json_path)
        label_idx = self.label_to_idx[label]
        if self.transform:
            keypoints = self.transform(keypoints)
        # Convert to tensors.
        keypoints = torch.tensor(keypoints, dtype=torch.float)
        label_idx = torch.tensor(label_idx, dtype=torch.long)
        return keypoints, label_idx

In [ ]:
# Cell 3: Aktualisierter STCGN-Modellcode
class STCGN(nn.Module):
    def __init__(self, num_keypoints, keypoint_dim, num_classes):
        super(STCGN, self).__init__()
        # Erste Convolution: Eingangsformat: (batch, keypoint_dim, seq_length, num_keypoints)
        self.conv1 = nn.Conv2d(in_channels=keypoint_dim, out_channels=64, kernel_size=(3,1), padding=(1,0))
        self.bn1 = nn.BatchNorm2d(64)
        # Zweiter Convolution-Block
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,1), padding=(1,0))
        self.bn2 = nn.BatchNorm2d(128)
        # Dritter Convolution-Block
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3,1), padding=(1,0))
        self.bn3 = nn.BatchNorm2d(256)
        # Dropout erhöhen
        self.dropout = nn.Dropout(0.6)
        # Adaptive Average Pooling: Reduziert die Zeitdimension
        self.avg_pool = nn.AdaptiveAvgPool2d((1, num_keypoints))
        # Vollständig verbundene Schicht
        self.fc = nn.Linear(256 * num_keypoints, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        # x: (batch, seq_length, num_keypoints, keypoint_dim)
        x = x.permute(0, 3, 1, 2)  # -> (batch, keypoint_dim, seq_length, num_keypoints)
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.dropout(x)
        x = self.avg_pool(x)  # -> (batch, 256, 1, num_keypoints)
        x = x.reshape(x.size(0), -1)  # -> (batch, 256 * num_keypoints)
        x = self.fc(x)
        return x

In [ ]:
# Cell 4: Define training and evaluation functions
def train_model(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(dataloader.dataset)

def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return running_loss / len(dataloader.dataset), correct / total


In [ ]:
# Cell 5: Aktualisierter Main-Block
def main():
    json_dir = '/home/haggenmueller/asl_detection/machine_learning/datasets/wlasl/keypoints'  # Pfad zu deinen JSON-Dateien
    batch_size = 32
    max_seq_length = 100
    num_epochs = 30  # Mehr Epochen
    dataset = ASLDataset(json_dir, max_seq_length=max_seq_length)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Bestimme die Eingabedimensionen
    sample, _ = dataset[0]
    num_keypoints = sample.shape[1]
    keypoint_dim = sample.shape[2]
    num_classes = len(dataset.labels)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = STCGN(num_keypoints, keypoint_dim, num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    # Weight decay hinzugefügt
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)
    # Scheduler, der die Lernrate senkt, wenn der Test-Loss nicht sinkt
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

    for epoch in range(num_epochs):
        train_loss = train_model(model, train_loader, criterion, optimizer, device)
        test_loss, test_acc = evaluate_model(model, test_loader, criterion, device)
        scheduler.step(test_loss)
        print(f'Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}')
    
    torch.save(model.state_dict(), 'stcgn_model.pth')

if __name__ == '__main__':
    main()